# 讓我們來計算熱點吧


    流程大概是這樣:
    1.照樣先把文章讀進來  分成tuple存起來
    2.然後每讀到一個點 就去mariaDB加一次那個點的count

    因為懶  每次就直接import可能會用到的東西

In [1]:
#encoding=utf-8
import jieba
import pandas as pd
import csv
import geocoder
import pymysql
import math
import numpy as np

    下面比較詳細的解說我放在runningTemplateFianl裡面  這邊就直接跑了
    

In [2]:
jieba.load_userdict("./dict/Hokkaido.txt")
df = pd.read_csv('./content/Hakkaicocontent.csv')
tt=[]
with open('./dict/Hokkaido.txt',encoding="UTF-8") as f:
    for i in f:
      tt.append(i.strip())
A=set()
B=[]
for partcontent in df['content']:
    words=jieba.cut(partcontent)
    for word in words:
        for ltt in tt:
            if word == ltt:
                A.add(ltt)
    if len(A)>1:
        B.append(list(A))
    A=set()
    

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\HOME\AppData\Local\Temp\jieba.cache
Loading model cost 0.712 seconds.
Prefix dict has been built succesfully.


In [3]:
for inner in B:
    print(inner)

['層雲峽', '千歲空港', '旭川', '富良野', '阿寒湖', '網走監獄博物館']
['鄂霍次克海', '宗谷海峽', '庫頁島', '旭川']
['函館', '金森倉庫群']
['札幌', '函館']
['札幌', '旭川']
['函館山', '洞爺湖', '函館', '五稜郭']
['札幌', '根室', '旭川']
['札幌', '層雲峽', '小樽', '函館', '金森倉庫群']
['札幌', '旭川']
['小樽', '旭川']
['札幌', '小樽']
['札幌', '狸小路', '小樽', '根室', '北海道神宮']
['富良野', '札幌', '層雲峽', '小樽', '旭川']
['富良野', '札幌', '石狩川', '旭川']
['層雲峽', '屈斜路湖', '根室', '旭川', '札幌', '網走']
['鄂霍次克海', '屈斜路湖', '能取湖', '千歲空港', '旭川', '富良野', '摩周湖', '阿寒湖', '網走湖', '佐呂間湖', '札幌', '網走']
['函館', '松前城']
['札幌', '羊之丘展望台']
['鄂霍次克海', '能取湖', '網走湖', '網走監獄博物館', '網走']
['函館山', '函館']
['富良野', '層雲峽', '旭川']
['小樽', '函館', '旭川']
['札幌', '小樽', '旭川']
['札幌', '函館', '旭川']
['札幌', '旭川', '網走']
['富良野', '旭川']
['狸小路', '小樽', '函館', '熊牧場']
['登別地獄谷', '小樽', '昭和新山', '舊道廳', '千歲空港', '熊牧場', '洞爺湖', '時計台', '函館山', '金森倉庫群', '札幌', '函館', '函館空港']
['小樽', '五稜郭公園', '洞爺湖', '函館山', '函館', '五稜郭']
['富良野', '庫頁島', '旭川']
['小樽', '五稜郭公園', '洞爺湖', '支笏', '函館山', '函館']
['五稜郭公園', '洞爺湖', '小樽', '函館', '函館山']
['札幌', '小樽', '函館']
['札幌', '狸小路', '小樽', '旭川']
['富良野', '札幌', '函館', '旭川']
['富良

    準備來計算次數了
    

In [10]:
db = pymysql.connect("1.162.65.171","bdse0305","bdse0305","bdse03-travel",charset='utf8' )
cur=db.cursor() 

In [11]:
for tmp in B:
    for inner in tmp:
        sql='SELECT * FROM ginwaydu WHERE place="'+inner+'"'
        if cur.execute(sql)==1:
            sql2 = "UPDATE ginwaydu  SET count = count + 1  WHERE place='"+inner+"'"
            cur.execute(sql2)
db.commit()
db.close()

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
